# Capstone Project

Metro Atlanta is the most popular metro area in Georgia and the ninth-largest metropolitan statistical area in the United States. It is home to nearly 6 millions people and more than 150,000 business, including The Home Depot, Coca-Cola company, Delta Air Lines, and UPS...

Variety of food choice make Atlanta charming. We can find out almost all types of food in this city: Thai food, Chinese food, Japanese. 

In this project, I will make some recommendation for foodies to pick the best county to live in metro Atlanta.

* **Geograpical coordinate of counties in Metro Atlanta**

In [1]:
from geopy.geocoders import Nominatim
import pandas as pd

In [6]:
county = ['Fulton, GA', 'DeKalb, GA', 'Gwinnett, GA', 'Cobb, GA', 'Clayton, GA', 'Coweta, GA', 'Douglas, GA', 'Fayette, GA', 'Henry, ,GA']
lat = []
lng = []
for i in range(len(county)):
    geolocator = Nominatim(user_agent='atl_explore')
    location = geolocator.geocode(county[i])
    lat.append(location.latitude)
    lng.append(location.longitude)

In [7]:
df = pd.DataFrame({'County':county, 'Latitude': lat, 'Longitude': lng})

In [9]:
county = df['County'].values
name = []
for i in range(len(county)):
    name.append(county[i].split(',')[0])

In [10]:
df.index = name

In [11]:
df

,County,Latitude,Longitude
Fulton,"Fulton, GA",34.058039,-84.296128
DeKalb,"DeKalb, GA",33.757561,-84.218651
Gwinnett,"Gwinnett, GA",33.956687,-84.022747
Cobb,"Cobb, GA",33.937395,-84.573201
Clayton,"Clayton, GA",33.520496,-84.359171
Coweta,"Coweta, GA",33.350066,-84.754573
Douglas,"Douglas, GA",33.699556,-84.762297
Fayette,"Fayette, GA",33.389927,-84.495376
Henry,"Henry, ,GA",33.451994,-84.136777


* **Restaurants in each counties**

In [12]:
import requests
from pandas.io.json import json_normalize 

In [13]:
# account information
CLIENT_ID = 'KGL3AWGZJOTUDQJ1OH03XE3XJD1YAYJVS55Y5EW14IM4DCDO' 
CLIENT_SECRET = 'X3NKM45152MTWIGHWNPJUYZKUVEKBWOM3VJCEZ0VX4RD4DAA' 
VERSION = '20180605' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KGL3AWGZJOTUDQJ1OH03XE3XJD1YAYJVS55Y5EW14IM4DCDO
CLIENT_SECRET:X3NKM45152MTWIGHWNPJUYZKUVEKBWOM3VJCEZ0VX4RD4DAA


In [92]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, VERSION, df['Latitude']['Fulton'], df['Longitude']['Fulton'], 50000, 500)
result = requests.get(url).json()
venue = result['response']['groups'][0]['items'] 
#venue_county = json_normalize(venue)
#venue_county.columns = ['Name', 'Categories', 'Latitude', 'Longitude']
#venue_county

Funtion to get venues

In [93]:
def get_venues(county):
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, CLIENT_SECRET, VERSION, df['Latitude'][county], df['Longitude'][county], 50000, 500)
    result = requests.get(url).json()
    venue = result['response']['groups'][0]['items'] 
    venue_county = json_normalize(venue).loc[:,['venue.name', 'venue.categories', 'venue.location.city', 'venue.location.lat', 'venue.location.lng']]
    venue_county.columns = ['Name', 'Categories', 'City', 'Latitude', 'Longitude']
    for i in range(venue_county.shape[0]):
        venue_county['Categories'][i]= venue_county['Categories'][i][0]['name']
    return(venue_county)
      

In [104]:
#county_value = [df['County'][i].split(',')[0] for i in range(df.shape[0])]
venue_all = pd.DataFrame(columns = ['Name', 'Categories', 'City', 'Latitude', 'Longitude'])
for i in county_value:
    venue = get_venues(i)
    venue_all = venue_all.append(venue, ignore_index = True)
    

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [105]:
venue_all['City'].unique()

array(['Alpharetta', 'Roswell', 'Johns Creek', 'Milton', 'Norcross',
       'Duluth', 'Peachtree Corners', 'Atlanta', 'Dunwoody', 'Marietta',
       'Cumming', 'Sandy Springs', 'Doraville', 'Brookhaven', 'Suwanee',
       'Woodstock', 'Lawrenceville', 'Powers Ferry Vantage', 'Smyrna',
       'Kennesaw', 'Decatur', 'Stone Mountain', 'Lithonia', 'Snellville',
       'Buford', 'Dacula', 'Conyers', 'Austell', 'Acworth',
       'Lithia Springs', 'Riverdale', 'Hampton', 'Fayetteville',
       'East Point', 'McDonough', 'Peachtree City', 'Newnan',
       'Sharpsburg', 'Senoia', 'Tyrone', 'Palmetto', 'Fairburn',
       'Union City', 'Carrollton', 'La Grange', 'Douglasville',
       'Villa Rica', 'College Park', nan, 'Jonesboro', 'Griffin',
       'Morrow', 'Locust Grove', 'Stockbridge', 'Forest Park',
       'Hapeville', 'Covington'], dtype=object)

In [106]:
venue_all.dropna(inplace = True)

In [108]:
venue_all.drop_duplicates(keep = 'first', inplace = True, ignore_index = True)

In [111]:
venue_all.shape

(414, 5)

## Output and save

In [121]:
venue_all.to_csv('/Users/chen-tze/Desktop/ATL_venue.csv')

In [122]:
df.to_csv('/Users/chen-tze/Desktop/ATL_county.csv')

In [119]:
venue_all.groupby('City')['Name'].count()

City
Acworth                   3
Alpharetta               12
Atlanta                 155
Austell                   1
Brookhaven                4
Buford                    2
Carrollton                1
College Park              3
Conyers                   4
Covington                 1
Cumming                   3
Dacula                    1
Decatur                  18
Doraville                 2
Douglasville              2
Duluth                    5
Dunwoody                  4
East Point                3
Fairburn                  4
Fayetteville             25
Forest Park               2
Griffin                   1
Hampton                   3
Hapeville                 2
Johns Creek               3
Jonesboro                 1
Kennesaw                  5
La Grange                 1
Lawrenceville             3
Lithia Springs            1
Lithonia                  2
Locust Grove              3
Marietta                 13
McDonough                 7
Milton                    1
Morrow         

* Cluster

In [113]:
venue_dummy = pd.get_dummies(venue_all['Categories'])
venue_dummy.head()

,Accessories Store,Airport Lounge,American Restaurant,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Dealership,...,Theme Park,Thrift / Vintage Store,Trail,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [114]:
venue_dummy.sum().sort_values(ascending = False)

Fast Food Restaurant         28
Pizza Place                  20
Trail                        19
American Restaurant          18
Grocery Store                16
                             ..
Middle Eastern Restaurant     1
Mountain                      1
Music Venue                   1
National Park                 1
Accessories Store             1
Length: 135, dtype: int64

In [115]:
import folium

In [90]:
geolocator = Nominatim(user_agent='atl_explore')
location = geolocator.geocode('Atlanta, GA')
atl_lat = location.latitude
atl_lng = location.longitude

In [112]:
# create map of Manhattan using latitude and longitude values
map_atl = folium.Map(location=[atl_lat, atl_lng], zoom_start=13)

# add markers to map
for lat, lng in zip(venue_all['Latitude'], venue_all['Longitude']):
    #label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        #popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_atl)  
    
map_atl

## Duluth

In [32]:
address = 'Duluth, GA'

geolocator = Nominatim()
location = geolocator.geocode(address)
lat_du = location.latitude
lng_du = location.longitude
print('The geograpical coordinate of Atlanta are {}, {}.'.format(lat_du, lng_du))

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Atlanta are 34.0028569, -84.1441046.


In [33]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
CLIENT_ID, CLIENT_SECRET, VERSION,lat_du, lng_du, 5000, 500)

result_du = requests.get(url).json()
venue_du = result_du['response']['groups'][0]['items']

In [34]:
duluth_venue = json_normalize(venue_du)
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
duluth_venue =duluth_venue.loc[:, filtered_columns]


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


In [36]:
# create map of Manhattan using latitude and longitude values
map_duluth = folium.Map(location=[lat_du, lng_du], zoom_start=13)

# add markers to map
for lat, lng in zip(duluth_venue['venue.location.lat'], duluth_venue['venue.location.lng']):
    #label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        #popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_duluth)  
    
map_duluth

## Doraville

In [37]:
address = 'Doraville, GA'

geolocator = Nominatim()
location = geolocator.geocode(address)
lat_do = location.latitude
lng_do = location.longitude
print('The geograpical coordinate of Atlanta are {}, {}.'.format(lat_do, lng_do))

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Atlanta are 33.8981579, -84.2832564.


In [38]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
CLIENT_ID, CLIENT_SECRET, VERSION,lat_do, lng_do, 5000, 500)

result_do = requests.get(url).json()
venue_do = result_do['response']['groups'][0]['items']

In [39]:
doraville_venue = json_normalize(venue_do)
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
doraville_venue =doraville_venue.loc[:, filtered_columns]


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


In [40]:
# create map of Manhattan using latitude and longitude values
map_doraville = folium.Map(location=[lat_do, lng_do], zoom_start=13)

# add markers to map
for lat, lng in zip(doraville_venue['venue.location.lat'], doraville_venue['venue.location.lng']):
    #label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        #popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_doraville)  
    
map_doraville